# 07 Hypothesis and Inference

In the classical setup, we have a null hypothesis,$H_0$ , that represents some default position, and some alternative hypothesis, $H_1$, that we’d like to compare it with. We use statistics to decide whether we can reject $H_0$ as false or not. This will probably make more sense with an example.

### Example: Flipping a Coin

Imagine we have a coin and we want to test whether it’s fair. We’ll make the assumption that the coin has some probability p of landing heads, and so our null hypothesis is that the coin is fair—that is, that p = 0.5. We’ll test this against the alternative hypothesis p ≠ 0.5.

In particular, our test will involve flipping the coin some number, n, times and counting the number of heads, X. Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable which we can approximate using the normal distribution:

In [1]:
from typing import Tuple
from typing import List

import math

def normal_approximation_to_binmomial(n:int, p:float) -> Tuple[float, float]:
    """Returns mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

Whenever a random variable follows a normal distribution, we can use
`normal_cdf` to figure out the probability that its realized value lies within
or outside a particular interval:

In [2]:
from scratchlib.probability import normal_cdf

# the normal cdf _is_ the probability the variable is below
# a threshold
normal_probability_below = normal_cdf

# its above the threhsold if its not belowe the threshold
def normal_probability_above(lo:float,
                             mu:float=0,
                             sigma:float=1) -> float:
    """The probability that an N(mu, sigma) is greater than lo."""
    return 1 - normal_cdf(lo, mu, sigma)

# its between if its less than hi, but not less than lo
def normal_probability_between(lo:float,
                               hi:float,
                               mu:float=0,
                               sigma:float=1) -> float:
    """The probability that an N(mu, sigma) is between lo and hi."""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# its outside iff its not between
def normal_probability_outside(lo:float,
                               hi:float,
                               mu:float=0,
                               sigma:float=1) -> float:
    """The probability that an N(mu, sigma) is not between lo and hi."""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

We can also do the reverse—find either the nontail region or the (symmetric) interval around the mean that accounts for a certain level of likelihood. For example, if we want to find an interval centered at the mean and containing 60% probability, then we find the cutoffs where the upper and lower tails each contain 20% of the probability (leaving 60%):

In [3]:
from scratchlib.probability import inverse_normal_cdf

def normal_upper_bound(probability:float,
                       mu:float=0,
                       sigma:float=1) -> float:
    """Returns the z for which P(Z <= z) = probability."""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability:float,
                       mu:float=0,
                       sigma:float=1) -> float:
    """Returns the z for which P(Z >= z) = probability."""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability:float,
                            mu:float=0,
                            sigma:float=1) -> Tuple[float, float]:
    """Returns the symmetric (about the mean) bounds
    that contains the specified probability
    """
    tail_probability = (1 - probability) / 2
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # lower bound should have tail_ptobability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound
    

In [5]:
mu_0, sigma_0 = normal_approximation_to_binmomial(1000, 0.5)
print(mu_0, sigma_0)

500.0 15.811388300841896


We need to make a decision about __significance__ —how willing we are to make a __type 1 error (“false positive”)__, in which we reject $H_0$ even though it’s true. For reasons lost to the annals of history, this willingness is often set at 5% or 1%. Let’s choose 5%.

Consider the test that rejects $H_0$ if X falls outside the bounds given by:

In [6]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lower_bound, upper_bound)

469.01026640487555 530.9897335951244


Assuming p really equals 0.5 (i.e., $H_0$ is true), there is just a 5% chance we observe an $X$ that lies outside this interval, which is the exact significance we wanted. Said differently, if $H_0$ is true, then, approximately 19 times out of 20, this test will give the correct result.

We are also often interested in the __power__ of a test, which is the probability of __not__ making a __type 2 error (“false negative”)__, in which we fail to reject $H_0$
even though it’s false. In order to measure this, we have to specify what exactly being false means. (Knowing merely that p is not 0.5 doesn’t give us a ton of information about the distribution of X.) In particular, let’s check what happens if p is really 0.55, so that the coin is slightly biased toward heads.

In that case, we can calculate the power of the test with:


In [9]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(f'lo: {lo},  hi: {hi}')

# actual mu and sigma absed on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binmomial(1000, 0.55)
print(f'mu_1: {mu_1},  sigma_1: {sigma_1}')

# a type 2 error means we fail to reject the null hypothesis,
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

print(f'type II prob: {type_2_probability},  power: {power}')

lo: 469.01026640487555,  hi: 530.9897335951244
mu_1: 550.0,  sigma_1: 15.732132722552274
type II prob: 0.11345199870463285,  power: 0.8865480012953671


This is a more powerful test, since it no longer rejects $H_0$ when $X$ is below 469 (which is very unlikely to happen if $H_1$ is true) and instead rejects $H_0$ when X is between 526 and 531 (which is somewhat likely to happen if $H_0$ is true).


## p-Values

An alternative way of thinking about the preceding test involves p-values. Instead of choosing bounds based on some probability cutoff, we compute the probability—assuming $H_0$ is true—that we would see a value at least as extreme as the one we actually observed.

For our two-sided test of whether the coin is fair, we compute:

In [10]:
def two_sided_p_value(x:float, mu:float=0, sigma:float=1) -> float:
    """How likely are we to see a value at least as extreme as x
    (in either direction) if our values are from an N(mu, sigma)?
    """
    if x >= mu:
        # x is greater than the mean, so the tail is everything greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)


If we were to see 530 heads, we would compute:

In [12]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

Why did we use a value of 529.5 rather than using 530? This is what's called a continuity correction. It reflects the fact that `normal_probability_between(529.5, 530.5, mu_0, sigma_0)` is a better estimate of the probability of seeing 530 heads than
`normal_probability_between(530, 531, mu_0, sigma_0)` is.

Correspondingly, `normal_probability_above(529.5, mu_0, sigma_0)` is a better estimate
of the probability of seeing at least 530 heads. You may have noticed that we also used this in the code that produced.

One way to convince yourself that this is a sensible estimate is with a simulation:

In [16]:
import random

extreme_value_count = 0
for __ in range(0, 1000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                    for __ in range(0, 1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1

# p-value was 0.062 => ~62 extreme values out of 1000
assert 59 < extreme_value_count < 65, f'{extreme_value_count}'

AssertionError: 71

Since the p-value is greater than our 5% significance, we don’t reject the null. If we instead saw 532 heads, the p-value would be:

In [17]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

which is smaller than the 5% significance, which means we would reject the null. It’s the exact same test as before. It’s just a different way of approaching the statistics.
Similarly, we would have:

In [18]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

For our one-sided test, if we saw 525 heads we would compute:

In [20]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

which means we wouldn’t reject the null. If we saw 527 heads, the computation would be:

In [21]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

and we would reject the null.

## Confidence Intervals
We’ve been testing hypotheses about the value of the heads probability p, which is a parameter of the unknown “heads” distribution. When this is the case, a third approach is to construct a confidence interval around the observed value of the parameter.

For example, we can estimate the probability of the unfair coin by looking at the average value of the Bernoulli variables corresponding to each flip— 1 if heads, 0 if tails. If we observe 525 heads out of 1,000 flips, then we estimate p equals 0.525.


How confident can we be about this estimate? Well, if we knew the exact value of p, the central limit theorem (recall “The Central Limit Theorem”) tells us that the average of those Bernoulli variables should be approximately normal, with mean p and standard deviation:

In [24]:
# we don't now p, we instead use our estimate
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

print(sigma)

0.015791611697353755


This is not entirely justified, but people seem to do it anyway. Using the normal approximation, we conclude that we are “95% confident” that the following interval contains the true parameter p:

In [25]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

__NOTE__

This is a statement about the interval, not about p. You should understand it as the assertion that if you were to repeat the experiment many times, 95% of the time the “true” parameter (which is the same every time) would lie within the observed confidence interval (which might be different every time).

In particular, we do not conclude that the coin is unfair, since 0.5 falls within our confidence interval.
If instead we’d seen 540 heads, then we’d have:

In [26]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat* (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

Here, “fair coin” doesn’t lie in the confidence interval. (The “fair coin” hypothesis doesn’t pass a test that you’d expect it to pass 95% of the time if it were true.)


## p-Hacking

A procedure that erroneously rejects the null hypothesis only 5% of the time will—by definition—5% of the time erroneously reject the null hypothesis:

In [83]:
from typing import List

def run_experiment() -> List[bool]:
    """Flips a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for __ in range(0, 1000)]

def reject_fairness(experiment:List[bool]) -> bool:
    """Using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for __ in range(0, 1000)]
num_rejections = len([experiment for experiment in experiments
                                 if reject_fairness(experiment)])

assert num_rejections == 46, f'{num_rejections}'

What this means is that if you’re setting out to find “significant” results, you usually can. Test enough hypotheses against your dataset, and one of them will almost certainly appear significant. Remove the right outliers, and you can probably get your p-value below 0.05.

This is sometimes called p-hacking and is in some ways a consequence of the “inference from p-values framework.”

If you want to do good science, you should determine your hypotheses before looking at the data, you should clean your data without the hypotheses in mind, and you should keep in mind that p-values are not substitutes for common sense. 

## Example: Running an A/B Test

One of your primary responsibilities at DataSciencester is experience optimization, which is a euphemism for trying to get people to click on advertisements. One of your advertisers has developed a new energy drink targeted at data scientists, and the VP of Advertisements wants your help choosing between advertisement A (“tastes great!”) and advertisement B (“less bias!”).

Being a scientist, you decide to run an experiment by randomly showing site visitors one of the two advertisements and tracking how many people click on each one.
If 990 out of 1,000 A-viewers click their ad, while only 10 out of 1,000 B- viewers click their ad, you can be pretty confident that A is the better ad. But what if the differences are not so stark? Here’s where you’d use statistical inference.


Let’s say that $N_A$ people see ad A, and that $n_A$ of them click it. We can think of each ad view as a Bernoulli trial where $p_A$ is the probability that someone clicks ad A. Then (if $N_A$ is large, which it is here) we know that $n_A / N_A$
is approximately a normal random variable with mean and standard deviation
$ \sigma_A = \sqrt{p_A(1 - p_A) / N_A} $

Similarly, $n_B / N_B$ is approximately a normal random variable with mean
$p_B$ and standard deviation $ \sigma_B = \sqrt{p_B (1 - p_B) / N_B} $

in code as:

In [85]:
def estimated_parameters(N:int, n:int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma



If we assume those two normals are independent (which seems reasonable, since the individual Bernoulli trials ought to be), then their difference should also be normal with mean $p_B - p_A$ and standard deviation $\sqrt{\sigma_A ^2 + \sigma_B ^2}$

This is sort of cheating. The math only works out exactly like this if you know the standard deviations. Here we’re estimating them from the data, which means that we really should be using a t-distribution. But for large enough datasets, it’s close enough that it doesn’t make much of a difference.

This means we can test the null hypothesis that $p_A$ and $p_B$ are the same (that is, that is 0) by using the statistic:

In [86]:
def a_b_test_statistic(N_A:int, n_A:int, N_B:int, n_B:int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

which should approximately be a standard normal.

For example, if “tastes great” gets 200 clicks out of 1,000 views and “less bias” gets 180 clicks out of 1,000 views, the statistic equals:

In [87]:
z = a_b_test_statistic(1000, 200, 1000, 180)

The probability of seeing such a large difference if the means were actually equal would be:

In [88]:
two_sided_p_value(z)

0.254141976542236

which is large enough that we can’t conclude there’s much of a difference. On the other hand, if “less bias” only got 150 clicks, we’d have:


In [89]:
z = a_b_test_statistic(1000, 200, 1000, 150)
two_sided_p_value(z)

0.003189699706216853

which means there’s only a 0.003 probability we’d see such a large difference if the ads were equally effective.m

## Bayesian Inference

The procedures we’ve looked at have involved making probability statements about our tests: e.g., “There’s only a 3% chance you’d observe such an extreme statistic if our null hypothesis were true.”


An alternative approach to inference involves __treating the unknown parameters themselves as random variables__. The analyst (that’s you) starts with a _prior distribution_ for the parameters and then uses the observed data and Bayes’s theorem to get an _updated posterior distribution_ for the parameters. __Rather than making probability judgments about the tests, you make probability judgments about the parameters.__



For example, when the unknown parameter is a probability (as in our coin- flipping example), we often use a prior from the Beta distribution, which puts all its probability between 0 and 1:

In [91]:
def B(alpha:float, beta:float) -> float:
    """A normalizing constant so that the total prob is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x:float, alpha:float, beta:float) -> float:
    if x <= 0 or x >= 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Generally speaking, this distribution centers its weight at:

```python
alpha / (alpha + beta)
```

and the larger and are, the “tighter” the distribution is.

For example, if and are both 1, it’s just the uniform distribution (centered at 0.5, very dispersed). If is much larger than , most of the weight is near 1. And if is much smaller than , most of the weight is near 0.

Say we assume a prior distribution on p. Maybe we don’t want to take a stand on whether the coin is fair, and we choose and to both equal 1. Or maybe we have a strong belief that the coin lands heads 55% of the time, and we choose equals 55, equals 45.

Then we flip our coin a bunch of times and see h heads and t tails. Bayes’s theorem (and some mathematics too tedious for us to go through here) tells us that the posterior distribution for p is again a Beta distribution, but with parameters and .

NOTE:
    
It is no coincidence that the posterior distribution was again a Beta distribution. The number of heads is given by a Binomial distribution, and the Beta is the conjugate prior to the Binomial distribution. This means that whenever you update a Beta prior using observations from the corresponding binomial, you will get back a Beta posterior.

Let’s say you flip the coin 10 times and see only 3 heads. If you started with the uniform prior (in some sense refusing to take a stand about the coin’s fairness), your posterior distribution would be a Beta(4, 8), centered around 0.33. Since you considered all probabilities equally likely, your best guess is close to the observed probability.
If you started with a Beta(20, 20) (expressing a belief that the coin was roughly fair), your posterior distribution would be a Beta(23, 27), centered around 0.46, indicating a revised belief that maybe the coin is slightly biased toward tails.

And if you started with a Beta(30, 10) (expressing a belief that the coin was biased to flip 75% heads), your posterior distribution would be a Beta(33, 17), centered around 0.66. In that case you’d still believe in a heads bias, but less strongly than you did initially.

If you flipped the coin more and more times, the prior would matter less and less until eventually you’d have (nearly) the same posterior distribution no matter which prior you started with.

For example, no matter how biased you initially thought the coin was, it would be hard to maintain that belief after seeing 1,000 heads out of 2,000 flips (unless you are a lunatic who picks something like a Beta(1000000,1) prior).

What’s interesting is that this allows us to make probability statements about hypotheses: “Based on the prior and the observed data, there is only a 5% likelihood the coin’s heads probability is between 49% and 51%.” This is philosophically very different from a statement like “If the coin were fair, we would expect to observe data so extreme only 5% of the time.”

Using Bayesian inference to test hypotheses is considered somewhat controversial—in part because the mathematics can get somewhat complicated, and in part because of the subjective nature of choosing a prior. 